In [4]:
import pandas as pd
import numpy as np
import sys
import os
import itertools
from sys import stdin
from IPython.display import display
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
dfloan = pd.read_csv('2012_to_2014_loans_data.csv')
dfinst = pd.read_csv('2012_to_2014_institutions_data.csv')

/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (3,4,5,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def Loan_Amount_Grouping(l):  # group loan amount 
    if   l <= 99: return 1
    elif l <= 136: return 2
    elif l <= 169: return 3
    elif l <= 200: return 4
    elif l <= 232: return 5
    elif l <= 270: return 6
    elif l <= 310: return 7
    elif l <= 370: return 8
    elif l <= 440: return 9
    elif l <= 800: return 10
    else: return 11


In [6]:
def hmda_init():
    dfloan = pd.read_csv('2012_to_2014_loans_data.csv')
    dfinst = pd.read_csv('2012_to_2014_institutions_data.csv')
    dfl = dfloan.copy()
    dfi = dfinst.copy()
    
    dfl['Respondent_ID'] = dfloan['Respondent_ID'].map(lambda x:format(x,'010d') if type(x) is not str else x)
    dfl['Census_Tract_Number'] = dfloan['Census_Tract_Number'].map(lambda x:x if type(x) is not str else (float(x) if not 'NA' in x else 0))
    dfl['Tract_to_MSA_MD_Income_Pct'] = dfloan['Tract_to_MSA_MD_Income_Pct'].map(lambda x:x if type(x) is not str else (float(x) if not 'NA' in x else 0))
    dfl['County_Code'] = dfloan['County_Code'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['FFIEC_Median_Family_Income'] = dfloan['FFIEC_Median_Family_Income'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['MSA_MD'] = dfloan['MSA_MD'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['Number_of_Owner_Occupied_Units'] = dfloan['Number_of_Owner_Occupied_Units'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))
    dfl['Applicant_Income_000'] = dfloan['Number_of_Owner_Occupied_Units'].map(lambda x:x if type(x) is not str else (int(x) if not 'NA' in x else 0))

    dfi['Respondent_Name_TS']=dfi['Respondent_Name_TS'].map(lambda x: x.replace(',',''))
    dfi['Respondent_Name_TS']=dfi['Respondent_Name_TS'].map(lambda x: x.replace('.',''))
    dfi['Respondent_Name_TS']=dfi['Respondent_Name_TS'].map(lambda x: ' '.join(x.split()))
    
    dfl['idx']=dfl['As_of_Year'].apply(str)+dfl['Respondent_ID'].apply(str)+dfl['Agency_Code'].apply(str)
    dfinst['idx']=dfinst['As_of_Year'].apply(str)+dfinst['Respondent_ID'].apply(str)+dfinst['Agency_Code'].apply(str)
    dfl['Respondent_Name'] = dfl['idx'].map(dfinst.set_index('idx')['Respondent_Name_TS'])
    
    dfl['Loan_Amount_Group'] = dfl['Loan_Amount_000'].map(Loan_Amount_Grouping)
    
    return dfl

In [10]:
y = pd.DataFrame({'a':dfloan['Loan_Amount_000']})

In [11]:

ranges = [0,99,136,169,200,232,270,310,370,440,800,100000]
y.groupby(pd.cut(y.a, ranges)).count()

KeyboardInterrupt: 

In [12]:
df= hmda_init()

/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2975: DtypeWarning: Columns (3,4,5,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [15]:
def hmda_to_json(data, states = 0, conventional_conforming = 0):
    if type(states) == list:
        df = data[data['State'].isin(states)]
    else:
        df=data.copy()
    
    if conventional_conforming != 0:
        df = df[df['Conventional_Conforming_Flag'] == conventional_conforming]
    
    df.to_json('hmda.json')
    return

In [88]:
hmda_to_json(df,['WV'],'Y')

In [89]:
df2=pd.read_json('hmda.json')

In [99]:
dfi=dfinst.copy()

In [122]:
dfinst.Respondent_Name_TS.unique()

array(['FIRST FINANCIAL BANK NA', 'FIRST FINANCIAL BANK, N.A.',
       'FIRST NATIONAL BANK OF GERMANT', 'EXCHANGE NATIONAL BANK & TRUST',
       'FIRST NATIONAL BANK AND TRUST', 'FNB BANK, N.A.',
       'THE 1ST NATIONAL BANK OF LACON', 'THE FIRST NATIONAL BANK OF SUF',
       'THE HONESDALE NATIONAL BANK', 'DNB FIRST, NA',
       'NATIONAL GRAND BANK', 'THE MILFORD NATIONAL BANK',
       'THE NATIONAL UNION BANK OF KIN', 'PUTNAM COUNTY NATIONAL BANK',
       'THE LYONS NATIONAL BANK', 'FIRST NATIONAL BANK OF GROTON',
       'ALLIANCE BANK NA', 'FIRST NATIONAL BANK OF OTTAWA',
       'ROCKPORT NATIONAL BANK', 'NATIONAL BANK OF MIDDLEBURY',
       'BALLSTON SPA NATIONAL BANK', 'NBT BANK NA',
       'COMMUNITY NATIONAL BANK', 'NATIONAL BANK OF COXSACKIE',
       'LANDMARK BANK', 'FIRST SOUTHERN NATIONAL BANK',
       'FIRST NATIONAL BANK SANTA FE', 'BOONE COUNTY NATIONAL BANK',
       'FIRST NATIONAL BANK MINNESOTA', 'FIRST NATIONAL BANK',
       'NATIONAL BANK & TRUST COMPANY', 'VINTON

In [33]:
df.head(1)

,Agency_Code,Applicant_Income_000,As_of_Year,Census_Tract_Number,County_Code,FFIEC_Median_Family_Income,Loan_Amount_000,MSA_MD,Number_of_Owner_Occupied_Units,Respondent_ID,Sequence_Number,State_Code,Tract_to_MSA_MD_Income_Pct,MSA_MD_Description,Loan_Purpose_Description,Agency_Code_Description,Lien_Status_Description,Loan_Type_Description,State,County_Name,Conforming_Limit_000,Conventional_Status,Conforming_Status,Conventional_Conforming_Flag,idx,Respondent_Name,Loan_Amount_Group
0,3,1545,2012,207.0,79,54900,94,16620,1545,0000000384,390,54,93.07,"CHARLESTON, WV",Refinance,FDIC,First Lien,Conventional,WV,PUTNAM,417.0,Conventional,Conforming,Y,201200000003843,THE OHIO VALLEY BANK COMPANY,1


In [41]:
len(df['Tract_to_MSA_MD_Income_Pct'].unique())

5250

In [36]:
df['FFIEC_Median_Family_Income'].unique()

array([ 54900,      0,  53000,  65900,  50300,  56600,  53200,  51700,
       105700,  51100,  65300,  45400,  63200,  81900,  62600,  85600,
       113400,  64400,  72600,  60500,  77800,  48700,  60700,  50200,
        60100,  75600,  70900,  52600,  57900,  53300,  66900,  50800,
        57400, 105900,  65700,  46100,  68000,  79300,  57500, 112200,
        59300,  78800,  77500,  45300,  59400,  73900,  63600,  73300,
        51600,  49900,  54100,  63500,  52700,  61000,  54600, 107100,
        52500,  56200,  67600,  47100,  62400,  64200,  83500,  90100,
       111300,  67300,  50600,  76400,  62900,  62700,  72200,  52000])

In [37]:
df['MSA_MD'].unique()

array([16620, 19060, 25180, 26580, 34060, 37620, 44600, 47894, 48540,
       49020,     0, 20100, 48864, 12580, 13644, 41540, 13980, 16820,
       19260, 25500, 28700, 31340, 40060, 40220, 47260, 13220, 48260,
       15680, 43524, 44420])

In [39]:
df['County_Code'].unique()

array([ 79,  43,  39,   5,  15,  57,   3,  65,  99,  11,  61,  77, 107,
        73, 105,   9,  29,  37,  69,  51,  27,  33,  83,  49,  97,  91,
        17,  63,   7,   1,  53,  35,  75,  25,   0,  95, 103,  85,  67,
        13,  41, 101,  93,  45,  81,  19, 109,  21,  87,  89,  31,  71,
        23,  55,  59,  47, 510, 121, 155, 750, 540, 125, 143, 590, 165,
       660, 191, 520, 169, 680, 515, 127, 760, 183, 149, 570, 670, 730,
        36, 145, 775, 161, 770, 550, 810, 710, 199, 735, 700, 650, 830,
       740, 800, 181, 115, 177, 600, 153, 179, 610, 187, 683, 685, 630,
       840, 137, 171, 197, 175, 117, 139, 193, 133, 195, 620, 167, 790,
       111, 157, 163, 141, 530, 119, 135, 820, 173, 640, 185, 690, 678,
       113, 131, 159, 147, 720, 580, 595])

In [42]:
len(df['Census_Tract_Number'].unique())

3019

In [58]:
df[df['Loan_Amount_000']<=10]

,Agency_Code,Applicant_Income_000,As_of_Year,Census_Tract_Number,County_Code,FFIEC_Median_Family_Income,Loan_Amount_000,MSA_MD,Number_of_Owner_Occupied_Units,Respondent_ID,Sequence_Number,State_Code,Tract_to_MSA_MD_Income_Pct,MSA_MD_Description,Loan_Purpose_Description,Agency_Code_Description,Lien_Status_Description,Loan_Type_Description,State,County_Name,Conforming_Limit_000,Conventional_Status,Conforming_Status,Conventional_Conforming_Flag,idx,Respondent_Name,Loan_Amount_Group
278,3,1103,2012,9580.00,15,54900,9,16620,1103,0000002444,64,54,71.82,"CHARLESTON, WV",Refinance,FDIC,First Lien,Conventional,WV,CLAY,417.0,Conventional,Conforming,Y,201200000024443,CLAY COUNTY BANK,1
320,5,1136,2012,131.00,39,54900,10,16620,1136,0000003576,56,54,84.49,"CHARLESTON, WV",Refinance,NCUA,Subordinate Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000035765,PIONEER WV FCU,1
349,5,1267,2012,108.02,39,54900,5,16620,1267,0000003576,146,54,99.47,"CHARLESTON, WV",Refinance,NCUA,Subordinate Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000035765,PIONEER WV FCU,1
388,5,1486,2012,21.00,39,54900,5,16620,1486,0000003576,230,54,151.96,"CHARLESTON, WV",Purchase,NCUA,Subordinate Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000035765,PIONEER WV FCU,1
390,5,1193,2012,6.00,39,54900,8,16620,1193,0000003576,236,54,81.87,"CHARLESTON, WV",Refinance,NCUA,Subordinate Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000035765,PIONEER WV FCU,1
1130,1,1805,2012,123.00,39,54900,3,16620,1805,0000014807,620,54,94.34,"CHARLESTON, WV",Refinance,OCC,First Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000148071,CITY NATIONAL BANK OF WV,1
1133,1,1587,2012,122.00,39,54900,7,16620,1587,0000014807,649,54,75.26,"CHARLESTON, WV",Refinance,OCC,First Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000148071,CITY NATIONAL BANK OF WV,1
1139,1,230,2012,1.00,39,54900,7,16620,230,0000014807,685,54,56.62,"CHARLESTON, WV",Refinance,OCC,First Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000148071,CITY NATIONAL BANK OF WV,1
1142,1,1492,2012,107.01,39,54900,6,16620,1492,0000014807,824,54,127.96,"CHARLESTON, WV",Refinance,OCC,First Lien,Conventional,WV,KANAWHA,417.0,Conventional,Conforming,Y,201200000148071,CITY NATIONAL BANK OF WV,1
1150,1,2300,2012,204.00,79,54900,6,16620,2300,0000014807,890,54,135.56,"CHARLESTON, WV",Refinance,OCC,First Lien,Conventional,WV,PUTNAM,417.0,Conventional,Conforming,Y,201200000148071,CITY NATIONAL BANK OF WV,1


In [171]:
index = df['As_of_Year'].unique()
columns = df['State'].unique()    
values = ['total','count','avg','pct']

df1 = pd.DataFrame(index=index, columns=values)
dfstate = pd.concat([df1]*len(columns),axis=1,keys=columns)


In [172]:
for i,row in dfstate.iterrows():
    year_total = df[df['As_of_Year']==i]['Loan_Amount_000'].sum()
    for state in columns:
        total = df[df['As_of_Year']==i][df['State']==state]['Loan_Amount_000'].sum()
        count = df[df['As_of_Year']==i][df['State']==state]['Loan_Amount_000'].count()
        avg = round(total/float(count),2)
        pct = round(total/float(year_total),4)
        dfstate.set_value(i,state,[total,count,avg,pct])
        

/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [164]:
dfstate

WV                              DE                              DC  \
        total  count     avg     pct    total  count     avg     pct    total   
2012  6023641  29024  207.54  0.0377  5253819  25238  208.17  0.0329  8368582   
2013  5838763  28490  204.94  0.0438  4598409  22504  204.34  0.0345  7092372   
2014  4984216  20137  247.52  0.0553  2991961  13964  214.26  0.0332  5044787   

                                   MD                                VA  \
      count     avg     pct     total   count     avg     pct     total   
2012  20733  403.64  0.0523  54095591  194802   277.7  0.3383  86144960   
2013  17395  407.72  0.0532  43640475  160472  271.95  0.3272  72210009   
2014  11333  445.14  0.0559  28235685   97363     290  0.3131  48920527   

                              
       count     avg     pct  
2012  288091  299.02  0.5388  
2013  244096  295.83  0.5414  
2014  147516  331.63  0.5425

In [165]:
index = df['As_of_Year'].unique()
columns = df['MSA_MD'].unique()    
values = ['total','count','avg','pct']

df1 = pd.DataFrame(index=index, columns=values)
dfMSA = pd.concat([df1]*len(columns),axis=1,keys=columns)


In [169]:
for i,row in dfMSA.iterrows():
    year_total = df[df['As_of_Year']==i]['Loan_Amount_000'].sum()
    for MSA in columns:
        total = df[df['As_of_Year']==i][df['MSA_MD']==MSA]['Loan_Amount_000'].sum()
        count = df[df['As_of_Year']==i][df['MSA_MD']==MSA]['Loan_Amount_000'].count()
        if count==0:
            avg=0
        else:
            avg = round(total/float(count),2)
        pct = round(total/float(year_total),4)
        dfMSA.set_value(i,MSA,[total,count,avg,pct])

/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/tianhong/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [170]:
dfMSA

16620                         19060                          25180  \
        total count     avg     pct   total count     avg     pct    total   
2012   958165  5436  176.26   0.006  222699  1698  131.15  0.0014  1252872   
2013  1118920  5204  215.01  0.0084  281191  1647  170.73  0.0021  1217434   
2014   633472  2490  254.41   0.007  126432  1111   113.8  0.0014   772796   

                             26580                         34060        \
     count     avg     pct   total count     avg     pct   total count   
2012  6928  180.84  0.0078  357636  2105   169.9  0.0022  691753  2665   
2013  6747  180.44  0.0091  325496  2049  158.86  0.0024  498601  2565   
2014  4296  179.89  0.0086  662000  2621  252.58  0.0073  421285  1729   

                       37620                         44600                \
         avg     pct   total count     avg     pct   total count     avg   
2012  259.57  0.0043  310929  1812  171.59  0.0019   97994   962  101.86   
2013  194.39  0.0037  353067  1744  202.45  0.0026  138358   746  185.47   
2014  243.66  0.0047  199197  1166  170.84  0.0022       0     0       0   

                 47894                           48540                        \
         pct     total   count     avg     pct   total count     avg     pct   
2012  0.0006  68272559  197087  346.41   0.427  268277  1241  216.18  0.0017   
2013   0.001  55869052  160171  348.81  0.4189  402671  1122  358.89   0.003   
2014       0  37538749   96928  387.28  0.4163  328905   790  416.34  0.0036   

       49020                          0                              20100  \
       total count     avg     pct    total  count     avg     pct   total   
2012  877412  4158  211.02  0.0055  9636892  40452  238.23  0.0603  858084   
2013  834447  3733  223.53  0.0063  8203947  38127  215.17  0.0615  726056   
2014  524308  2352  222.92  0.0058  4571991  16638  274.79  0.0507  498139   

                              48864                            12580         \
     count     avg     pct    total  count     avg     pct     total  count   
2012  4564  188.01  0.0054  4000944  18900  211.69   0.025  23024594  87875   
2013  3931   184.7  0.0054  3469475  16676  208.05   0.026  19170624  73949   
2014  2561  194.51  0.0055  2104658   9703  216.91  0.0233  12463923  45109   

                         13644                           41540                \
         avg     pct     total  count     avg     pct    total count     avg   
2012  262.02   0.144  19263638  56613  340.27  0.1205   360130  2175  165.58   
2013  259.24  0.1437  14118049  41287  341.95  0.1058   323871  2033  159.31   
2014  276.31  0.1382         0      0       0       0  1069845  5299   201.9   

               13980                          16820                        \
         pct   total count     avg     pct    total count     avg     pct   
2012  0.0023  572336  3206  178.52  0.0036  1681980  6286  267.58  0.0105   
2013  0.0024  627140  2992  209.61  0.0047  1584904  5433  291.72  0.0119   
2014  0.0119  355612  1940  183.31  0.0039  1160587  3476  333.89  0.0129   

       19260                         25500                         28700  \
       total count     avg     pct   total count     avg     pct   total   
2012  214877  1263  170.13  0.0013  700268  2742  255.39  0.0044  296455   
2013  398573  1348  295.68   0.003  517194  2376  217.67  0.0039  296780   
2014       0     0       0       0  502925  1453  346.13  0.0056  127906   

                              31340                          40060         \
     count     avg     pct    total count     avg     pct    total  count   
2012  1498   197.9  0.0019  1105642  5406  204.52  0.0069  9529655  40031   
2013  1474  201.34  0.0022  1228236  5209  235.79  0.0092  8420322  36684   
2014   952  134.36  0.0014   736771  3226  228.39  0.0082  5688095  22466   

                        40220                           47260                 \
         avg     pct    total count     avg

In [173]:
df['Respondent_Name'].value_counts()

WELLS FARGO BANK, NA              137060
QUICKEN LOANS                      66561
BANK OF AMERICA, N.A.              48506
BRANCH BANKING AND TRUST CO        47543
SUNTRUST MORTGAGE, INC             46798
JPMORGAN CHASE BANK, NA            44010
NFCU                               33971
GEORGE MASON MORTGAGE LLC          29021
PNC BANK N.A.                      26117
CITIBANK, N.A.                     23885
FIRST HOME MORTGAGE                23239
USAA FEDERAL SAVINGS BANK          19949
MONARCH BANK                       17634
EMBRACE HOME LOANS, INC.           14218
FIRST SAVINGS MORTGAGE CORPORA     13985
FLAGSTAR BANK                      13513
PROSPERITY MORTGAGE COMPANY        13102
FIFTH THIRD MORTGAGE COMPANY       11945
1ST MARINER BANK                   11569
PRIMELENDING                       11410
FREEDOM MORTGAGE CORPORATION       11058
PRESIDENTIAL BANK, F.S.B.          10739
NVR MORTGAGE FINANCE INC           10703
TOWNEBANK                          10396
MCLEAN MORTGAGE 